In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sequential.models import NeuralNet
from sequential.layers import *
from sequential.preprocessing import preprocessing
from sequential.metrics import mean_absolute_error

### Load data

In [ ]:
df = pd.read_csv('../data/AirPassengers.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

### Preprocessing

In [ ]:
val_len = 12 # number of data points to reserve for validation
time_steps = 36 # number of time steps in each training sequence
target_time_steps = 12 # number of time steps to predict

X = df.to_numpy()
X_orig = X

# scale and turn X into sequences 
X, X_train, X_test, y_train, y_test, scaler = preprocessing(X, val_len, time_steps, target_time_steps)

# scale y (target variable) for plotting alongside scaled forecasts
y = scaler.transform(X_orig)[:, 0].flatten()

# print the training data shape: (num_sequences, time_steps, num_features)
X_train.shape, y_train.shape

### Model fit

In [ ]:
%%time

# set a random seed for repeatable results
# np.random.seed(7)

layers = [
    Dense(48, activation="relu", relu_alpha=.01),
    Dense(24, activation='relu', relu_alpha=.01),
    Dense(1, activation=None, use_bias=False),
]

optimizer = 'adam'
optimizer_args = {'alpha': .001}

model = NeuralNet(layers, optimizer=optimizer, optimizer_args=optimizer_args, loss='mae')

fitted_vals, loss = model.fit(X_train, y_train, epochs=2000, batch_size=None, verbose_rate=100)

### Fitted values plot

In [ ]:
plt.plot(y_train.flatten())
plt.plot(fitted_vals.flatten())
print('Fit MAE: ', mean_absolute_error(y_train.flatten(), fitted_vals.flatten()))

### Validation forecast

In [ ]:
forecast_steps = val_len
val_forecast = model.get_forecast(forecast_steps, X_test).flatten()
val_forecast

In [ ]:
plt.plot(y)
forecast_len = len(val_forecast)
# create an empty array with the forecast values at the end
forecast_array = np.zeros(len(y))
forecast_array[:] = np.inf
forecast_array[-forecast_len:] = val_forecast
plt.plot(forecast_array)

print('MAE:', mean_absolute_error(y_test, val_forecast[:val_len]))

In [ ]:
plt.plot(y_test)
plt.plot(val_forecast[:val_len])
print('MAE:', mean_absolute_error(y_test, val_forecast[:val_len]))

### Forecast beyond historical data

In [ ]:
forecast = model.get_forecast(forecast_steps, X[-1][np.newaxis, :, :]).flatten()

In [ ]:
plt.plot(y)
# create an empty array with the forecast values at the end
forecast_array = np.zeros(y.size + forecast.size)
forecast_array[:] = np.inf
forecast_array[-forecast.size:] = forecast
plt.plot(forecast_array)